In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CS598_DL4H'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
os.chdir(GOOGLE_DRIVE_PATH)

['mimic3-benchmarks-master', 'prepare_dataset.ipynb']


In [4]:
%cd mimic3-benchmarks-master

/content/drive/My Drive/CS598_DL4H/mimic3-benchmarks-master


In [5]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import copy
import io

import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn.functional as F

from utils import utils
from utils.readers import InHospitalMortalityReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils

In [6]:
data_path = 'data/hospital-mortality/'

In [19]:
def demographic(demo_path):
  demo = []
  diag = []
  idx = []
  for cur_name in os.listdir(demo_path):
    cur_id, cur_episode = cur_name.split('_', 1)
    cur_episode = cur_episode[:-4]
    cur_file = demo_path + cur_name

    with io.open(cur_file, 'r', encoding='utf-8', errors='ignore') as tsfile:
      header = tsfile.readline().strip().split(',')
      if header[0] != "Icustay":
        continue
      cur_data = tsfile.readline().strip().split(',')
          
      if len(cur_data) == 1:
        cur_demo = np.zeros(12)
        cur_diag = np.zeros(128)
      else:
        if cur_data[3] == '':
          cur_data[3] = 60.0
        if cur_data[4] == '':
          cur_data[4] = 160
        if cur_data[5] == '':
          cur_data[5] = 60

        cur_demo = np.zeros(12)
        cur_demo[int(cur_data[1])] = 1
        cur_demo[5 + int(cur_data[2])] = 1
        cur_demo[9:] = cur_data[3:6]
        cur_diag = np.array(cur_data[8:], dtype=np.int)

    demo.append(cur_demo)
    diag.append(cur_diag)
    idx.append(cur_id+'_'+cur_episode)
    
  return demo, diag, idx


demographic_data = []
diagnosis_data = []
idx_list = []
demo_folders = ['demo1/', 'demo2/', 'demo3/', 'demo4/', 'demo5/']
for d in demo_folders:
  demo_path = data_path + d
  print(demo_path)
  demo, diag, idx = demographic(demo_path)
  demographic_data.extend(demo)
  diagnosis_data.extend(diag)
  idx_list.extend(idx)


for each_idx in range(9,12):
  cur_val = []
  for i in range(len(demographic_data)):
    cur_val.append(demographic_data[i][each_idx])
  cur_val = np.array(cur_val)
  _mean = np.mean(cur_val)
  _std = np.std(cur_val)
  _std = _std if _std > 1e-7 else 1e-7
  for i in range(len(demographic_data)):
    demographic_data[i][each_idx] = (demographic_data[i][each_idx] - _mean) / _std

data/hospital-mortality/demo1/


<ipython-input-19-80cb4b130a79>:31: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cur_diag = np.array(cur_data[8:], dtype=np.int)


data/hospital-mortality/demo2/
data/hospital-mortality/demo3/
data/hospital-mortality/demo4/
data/hospital-mortality/demo5/


In [20]:
with open('demographic_data.txt', 'w') as f:
  for arr in demographic_data:
    np.savetxt(f, [arr])

# demographic_data = []
# with open('demographic_data.txt', 'r') as f:
#   for line in f:
#     demographic_data.append(np.fromstring(line, dtype=float, sep=' '))

In [21]:
with open('diagnosis_data.txt', 'w') as f:
  for arr in diagnosis_data:
    np.savetxt(f, [arr])

# diagnosis_data = []
# with open('diagnosis_data.txt', 'r') as f:
#   for line in f:
#     diagnosis_data.append(np.fromstring(line, dtype=int, sep=' '))

In [22]:
with open('idx_list.txt', 'w') as f:
  for string in idx_list:
    f.write(string + '\n')

# idx_list = []
# with open('idx_list.txt', 'r') as f:
#   for line in f:
#     idx_list.append(line.strip())

In [23]:
demographic_data

[array([ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.1809265 ,
        -0.23065275, -0.38474891]),
 array([ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.0239647 ,
        -0.23065275, -0.32019361]),
 array([ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  0.        , -1.74035025,
        -0.23065275, -0.38474891]),
 array([ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  0.        , -2.09045922,
        -0.23065275,  0.07359383]),
 array([ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.50439253,
        -0.23065275, -0.30297887]),
 array([ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.      

In [24]:
len(demographic_data)

42125